Code assumes that data is in same directory as capstone_repo directory:  
```
capstone_repo  
│   README.md  
│   gsc_mfcc_extraction.ipynb    
│  
speech_commands_v0.02  
└───backward  
│   │   file01.wav  
│   │   file02.wav (etc)  
└───bed  
│   |..etc  
```

In [2]:
#! pip install torch

     |████████████████████████████████| 129.9 MB 22.8 MB/s eta 0:00:01


In [4]:
#! pip install torchaudio

     |████████████████████████████████| 2.4 MB 3.3 MB/s eta 0:00:01


In [6]:
#import libraries
import torch
import numpy as np
import os
import IPython.display as ipd
import torchaudio
import torchaudio.functional as F
import torchaudio.transforms as T
import time
import pandas as pd

In [34]:
data_path = '/Users/Shared/back up fix/speech_commands_v0.02/'

In [8]:
# get list of word directories in speech commands dataset
list(os.walk(data_path))[0]

('/Users/Shared/back up fix/speech_commands_v0.02',
 ['right',
  'eight',
  'cat',
  'tree',
  'backward',
  'learn',
  'bed',
  'happy',
  'go',
  'dog',
  'no',
  'wow',
  'follow',
  'nine',
  'left',
  'stop',
  'three',
  '_background_noise_',
  'sheila',
  'one',
  'bird',
  'zero',
  'seven',
  'up',
  'visual',
  'marvin',
  'two',
  'house',
  'down',
  'six',
  'yes',
  'on',
  'five',
  'forward',
  'off',
  'four'],
 ['.DS_Store',
  'validation_list.txt',
  'LICENSE',
  'README.md',
  'testing_list.txt'])

In [9]:
all_words = ['backward',
  'bed',
  'bird',
  'cat',
  'dog',
  'down',
  'eight',
  'five',
  'follow',
  'forward',
  'four',
  'go',
  'happy',
  'house',
  'learn',
  'left',
  'marvin',
  'nine',
  'no',
  'off',
  'on',
  'one',
  'right',
  'seven',
  'sheila',
  'six',
  'stop',
  'three',
  'tree',
  'two',
  'up',
  'visual',
  'wow',
  'yes',
  'zero']

In [10]:
#only these 20 words will act as keywords
word_list = ['up','down','left', 'right', 'stop', 'go', 'yes', 'no', 'on', 'off', 'one', 'two', 
             'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'zero']
len(word_list)

20

In [11]:
#the other 15 words will be combined into one class of rejection words
rejection_words = [i for i in all_words if i not in word_list]
print(rejection_words)
len(rejection_words)

['backward', 'bed', 'bird', 'cat', 'dog', 'follow', 'forward', 'happy', 'house', 'learn', 'marvin', 'sheila', 'tree', 'visual', 'wow']


15

In [12]:
#! pip install librosa

     |████████████████████████████████| 213 kB 4.1 MB/s eta 0:00:01
     |████████████████████████████████| 377 kB 22.9 MB/s eta 0:00:01
     |████████████████████████████████| 323 kB 46.1 MB/s eta 0:00:01
     |████████████████████████████████| 56 kB 9.9 MB/s  eta 0:00:01
     |████████████████████████████████| 613 kB 141.2 MB/s eta 0:00:01
  Created wheel for audioread: filename=audioread-2.1.9-py3-none-any.whl size=23153 sha256=7c00f110a1cf6128f4b2ea161087c0aa98df2940b31ce9442b532f8ffc2c552f
  Stored in directory: /Users/JosephTomal_1/Library/Caches/pip/wheels/a2/a3/bd/ec1568ce7515115a11ab686d509ad302124c782af065de47ee
  Created wheel for resampy: filename=resampy-0.2.2-py3-none-any.whl size=320732 sha256=df51d709330238071fc39c57d8a25db49202e582f267be5c913d32e15d508bc1
  Stored in directory: /Users/JosephTomal_1/Library/Caches/pip/wheels/86/2c/7d/46a32a246b0e5939cea2c5ec1492164073e0c5d16d666ae2cd
Successfully built audioread resampy


In [13]:
import librosa

In [14]:
#test to see if one audio file will work correctly
y, sr = librosa.load('/Users/Shared/back up fix/speech_commands_v0.02/bird/0a7c2a8d_nohash_0.wav', sr = 16000)

In [18]:
#need a tensor as input not numpy array of time series audio
y = torch.from_numpy(y)

In [21]:
def quantize_data(data, num_bits=8):
              step_size = 2.0/2**num_bits
              max_val = 2**num_bits -1
              q_data = torch.round(data / step_size)
              q_data = torch.clamp(q_data, 0, max_val)
              q_data /= 256.
              q_data = q_data.sub(0.5).mul(256.).round().clamp(min=-128, max=127).div(128.)
              return q_data

In [44]:
#loop through all files, calculate MFCCs, save them to the appropriate directory
start = time.time()

for i in all_words:
    #move to the directory for the ith word
    working_path = data_path + "/" + i
    
    #loop through each file in the word's directory
    for j in os.listdir(working_path):
        #define paths we'll need
        word_path = i + '/' + j
        file_path = data_path + word_path
        
        #load audio file, GSC is sampled at 16000Hz
        y, sr = librosa.load(file_path, sr = 16000)
        
        # quantize data
        y = torch.from_numpy(y)
        
        quant = quantize_data(y, num_bits=8)
        
        #define save location directory
        #word_dir = i + '_quant'
        
        #saving file 
        #torch.save(y, 'Users/Shared/back up fix/quantized/%s.npy' % i)

end = time.time()
print(end - start)

52.561039686203


In [45]:
y

tensor([-3.0518e-05,  6.1035e-05, -3.0518e-05,  ..., -3.0518e-05,
        -3.0518e-05, -6.1035e-05])